# AutoML using H2O

Copyright @ 2020 **ABCOM Information Systems Pvt. Ltd.** All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and limitations under the License.

# Install H2O

In [ ]:
pip install h2o

     |████████████████████████████████| 129.4MB 87kB/s 
  Created wheel for h2o: filename=h2o-3.30.1.3-py2.py3-none-any.whl size=129446676 sha256=be8671ee11a75c73bf37775991bec65fa6f31eaa32801585419beda585bd2ba1
  Stored in directory: /root/.cache/pip/wheels/3d/e6/07/53ce9be9cb61b33a79cb3ed1fc39f2dae84f6ee6fe1e373e5e
Successfully built h2o


# Start H2O cluster

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9" 2020-10-20; OpenJDK Runtime Environment (build 11.0.9+11-Ubuntu-0ubuntu1.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.9+11-Ubuntu-0ubuntu1.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpt2kna8tz
  JVM stdout: /tmp/tmpt2kna8tz/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpt2kna8tz/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,1 month and 12 days
H2O_cluster_name:,H2O_from_python_unknownUser_dhpwhi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


# Part I

## Downloading data

In [ ]:
!wget https://github.com/abcom-mltutorials/AutoML-H2O/blob/main/rossmann-store-sales.zip?raw=true

--2020-11-10 12:34:20--  https://github.com/abcom-mltutorials/AutoML-H2O/blob/main/rossmann-store-sales.zip?raw=true
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/abcom-mltutorials/AutoML-H2O/raw/main/rossmann-store-sales.zip [following]
--2020-11-10 12:34:20--  https://github.com/abcom-mltutorials/AutoML-H2O/raw/main/rossmann-store-sales.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/abcom-mltutorials/AutoML-H2O/main/rossmann-store-sales.zip [following]
--2020-11-10 12:34:21--  https://raw.githubusercontent.com/abcom-mltutorials/AutoML-H2O/main/rossmann-store-sales.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.gith

In [ ]:
!unzip rossmann-store-sales.zip?raw=true

Archive:  rossmann-store-sales.zip?raw=true
  inflating: sample_submission.csv   
  inflating: store.csv               
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
#load data into H2O
traindf= h2o.import_file("/content/train.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
traindf.head()

Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
1,5,2015-07-31 00:00:00,5263,555,1,1,0,1
2,5,2015-07-31 00:00:00,6064,625,1,1,0,1
3,5,2015-07-31 00:00:00,8314,821,1,1,0,1
4,5,2015-07-31 00:00:00,13995,1498,1,1,0,1
5,5,2015-07-31 00:00:00,4822,559,1,1,0,1
6,5,2015-07-31 00:00:00,5651,589,1,1,0,1
7,5,2015-07-31 00:00:00,15344,1414,1,1,0,1
8,5,2015-07-31 00:00:00,8492,833,1,1,0,1
9,5,2015-07-31 00:00:00,8565,687,1,1,0,1
10,5,2015-07-31 00:00:00,7185,681,1,1,0,1


In [ ]:
traindf.describe()

Rows:1017209
Cols:9




,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
type,int,int,time,int,int,int,int,int,int
mins,1.0,1.0,1356998400000.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,558.4297268309653,3.998340557348601,1397179842846.0571,5773.818972305597,633.1459464082602,0.8301066939045958,0.3815145166824124,0.0,0.17864666946517382
maxs,1115.0,7.0,1438300800000.0,41551.0,7388.0,1.0,1.0,0.0,1.0
sigma,321.9086511434529,1.99739096494053,23712834841.056667,3849.926175234752,464.4117338866319,0.37553922469319473,0.48575860487742,0.0,0.38305636818092625
zeros,0,0,0,172871,172869,172817,629129,986159,835488
missing,0,0,0,0,0,0,0,31050,0
0,1.0,5.0,2015-07-31 00:00:00,5263.0,555.0,1.0,1.0,0.0,1.0
1,2.0,5.0,2015-07-31 00:00:00,6064.0,625.0,1.0,1.0,0.0,1.0
2,3.0,5.0,2015-07-31 00:00:00,8314.0,821.0,1.0,1.0,0.0,1.0


## Features and Target seletions

In [ ]:
features=['Store','DayOfWeek','Date','Open','Promo','StateHoliday','SchoolHoliday']
target='Sales'

## Running AutoML

In [ ]:
aml= H2OAutoML(max_models=10,max_runtime_secs=300,seed=1)
aml.train(x=features,y=target,training_frame=traindf)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Generating Leaderboard

In [ ]:
result = h2o.automl.get_leaderboard(aml, extra_columns = 'ALL')
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201110_123430,6.17895e+06,2485.75,6.17895e+06,1652.72,nan
StackedEnsemble_BestOfFamily_AutoML_20201110_123430,6.71158e+06,2590.67,6.71158e+06,1773.93,2.57259
DRF_1_AutoML_20201110_123430,7.86051e+06,2803.66,7.86051e+06,1787.9,0.389368
GBM_5_AutoML_20201110_123430,1.28956e+07,3591.05,1.28956e+07,2696.07,3.53902
GBM_3_AutoML_20201110_123430,1.30272e+07,3609.33,1.30272e+07,2711.43,3.53939
GBM_1_AutoML_20201110_123430,1.30671e+07,3614.85,1.30671e+07,2709.12,3.54002
GBM_2_AutoML_20201110_123430,1.30808e+07,3616.74,1.30808e+07,2710.43,3.54172
GBM_4_AutoML_20201110_123430,1.30888e+07,3617.85,1.30888e+07,2713.02,3.54471
GLM_1_AutoML_20201110_123430,1.4823e+07,3850.07,1.4823e+07,2887.75,3.5926
XGBoost_1_AutoML_20201110_123430,2.47678e+07,4976.72,2.47678e+07,3811.7,1.02356


## Inference

In [ ]:
testdf= h2o.import_file('/content/test.csv')
prediction = aml.predict(testdf)
print ("\nPredicted sales: ", prediction)

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%

Predicted sales:  

predict
7338.29
8507.2
8669.9
8093.49
8093.49
8093.49
8093.49
8096.65
8096.65
8096.65


# Part II

## Deep Learning Algorithm

## Import estimator

In [ ]:
# Import the Deep Learning Estimator
from h2o.estimators import H2ODeepLearningEstimator

## Loading data

In [ ]:
df= h2o.import_file("/content/train.csv")
train, test = df.split_frame([0.8], seed=42)

Parse progress: |█████████████████████████████████████████████████████████| 100%


## Model building

In [ ]:
dl = H2ODeepLearningEstimator(distribution="tweedie",
                               hidden=[5],
                               epochs=1000,
                               train_samples_per_iteration=-1,
                               input_dropout_ratio=0.1,
                               activation="Tanh",
                               single_node_mode=False,
                               score_training_samples=0,
                               score_validation_samples=0
                              )                   

## Model training

In [ ]:
features=['Store','DayOfWeek','Date','Open','Promo','StateHoliday','SchoolHoliday']
target='Sales'

In [ ]:
 dl.train(x=features,y=target,training_frame=train,validation_frame=test)

/usr/local/lib/python3.6/dist-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Dropping bad and constant columns: [StateHoliday]
  warnings.warn(mesg["message"], RuntimeWarning)


deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [ ]:
print(dl)

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_model_python_1605011655972_1


Status of Neuron Layers: predicting Sales, regression, tweedie distribution, Automatic loss, 41 weights/biases, 4.1 KB, 117,229,968 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,6,Input,10,,,,,,,,,
1,,2,5,Tanh,0,0,0,0.235821,0.422355,0,-1.33791,3.96536,4.42192,6.90047
2,,3,1,Linear,,0,0,6.39451e-06,1.01895e-07,0,1.60994,6.62218,10.2626,1.09713e-154




ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 6857417.442711691
RMSE: 2618.6671118551308
MAE: 1729.311314647968
RMSLE: 0.36796968993556445
Mean Residual Deviance: 10.558378472342577

ModelMetricsRegression: deeplearning
** Reported on validation data. **

MSE: 6910840.062842542
RMSE: 2628.847668246021
MAE: 1733.738939775577
RMSLE: 0.3694304793684597
Mean Residual Deviance: 10.62099367131425

Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_deviance,training_mae,training_r2,validation_rmse,validation_deviance,validation_mae,validation_r2
0,,2020-11-10 12:37:34,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-11-10 12:37:36,4.109 sec,432340 obs/sec,1.0,1,814097.0,2783.955263,12.093412,1787.568625,0.476981,2791.979172,12.144673,1791.131514,0.474551
2,,2020-11-10 12:37:57,24.567 sec,815767 obs/sec,21.0,21,17096037.0,2937.322651,13.371305,1924.484352,0.417768,2954.362851,13.487938,1931.948263,0.411653
3,,2020-11-10 12:38:12,38.544 sec,841804 obs/sec,35.0,35,28493395.0,2783.577786,12.160462,1882.852126,0.477123,2793.812044,12.229132,1887.696767,0.473861
4,,2020-11-10 12:38:23,49.632 sec,853759 obs/sec,46.0,46,37448462.0,2731.278894,11.281388,1813.121485,0.496586,2744.767997,11.350940,1819.368464,0.492171
5,,2020-11-10 12:38:34,1 min 0.744 sec,860950 obs/sec,57.0,57,46403529.0,2638.068718,10.756204,1741.094359,0.530360,2647.604930,10.812668,1745.469031,0.527488
6,,2020-11-10 12:38:45,1 min 11.652 sec,842050 obs/sec,66.0,66,53730402.0,2662.725591,10.902603,1778.454426,0.521540,2671.766283,10.955908,1783.247579,0.518825
7,,2020-11-10 12:38:55,1 min 21.915 sec,846694 obs/sec,76.0,76,61871372.0,2784.354234,12.282304,1837.859304,0.476831,2793.507914,12.342998,1841.301914,0.473976
8,,2020-11-10 12:39:05,1 min 32.186 sec,850242 obs/sec,86.0,86,70012342.0,2618.667112,10.558378,1729.311315,0.537242,2628.847668,10.620994,1733.738940,0.534160
9,,2020-11-10 12:39:16,1 min 42.282 sec,853817 obs/sec,96.0,96,78153312.0,2682.358122,11.072975,1822.452116,0.514458,2692.487779,11.135043,1828.048580,0.511332



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Open,1.000000,1.000000,0.644378
1,Promo,0.285726,0.285726,0.184115
2,DayOfWeek,0.136520,0.136520,0.087971
3,SchoolHoliday,0.055688,0.055688,0.035884
4,Date,0.044772,0.044772,0.028850
5,Store,0.029178,0.029178,0.018802


## Model evaluation

In [ ]:
#Evaluate the performance
dlperformance = dl.model_performance()
dlperformance


ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 6857417.442711691
RMSE: 2618.6671118551308
MAE: 1729.311314647968
RMSLE: 0.36796968993556445
Mean Residual Deviance: 10.558378472342577
